
# Update recipes

Below are some ad-hoc code chunks to assist me in mass-updating conda-forge R recipes

In [34]:
import os
import sys

In [2]:
# Authenticate with GitHub

import github

gh_token = os.environ['GH_TOKEN']
gh = github.Github(gh_token)
gh_me = gh.get_user()
print("Authenticated as %s"%(gh_me.login))

Authenticated as jdblischak


In [32]:
# Check rate limit
gh.get_rate_limit()

RateLimit(core=Rate(reset=2021-12-30 03:11:25, remaining=5000, limit=5000))

In [60]:
# Search
# https://pygithub.readthedocs.io/en/latest/github.html?highlight=search#github.MainClass.Github.search_code
results = gh.search_code(query="org:conda-forge jdblischak language:YAML MIT")
results.totalCount

33

In [48]:
# Repositories to keep
skip = ['r-feedstock',
        'r-base-feedstock',
        'r-git2r-feedstock',
        'r-knitr-feedstock',
        'r-rmarkdown-feedstock',
        'r-workflowr-feedstock']

In [75]:
# Convert search results to dict of repository objects
dict_repo = {}
for r in results:
    repo = r.repository
    name_repo = os.path.basename(repo.full_name)
    if name_repo in skip:
        continue
    if name_repo[:2] != "r-":
        continue
    dict_repo[name_repo] = repo
print(len(dict_repo))

26


In [101]:
# Save list of packages to generate with helper script
dir_helper = "../../conda_r_skeleton_helper"
if not os.path.exists(dir_helper):
    sys.stderr.write("Need to clone helper repo")
    sys.exit(1)
file_packages = dir_helper + "/packages.txt"
handle_packages = open(file_packages, "w")
for name_temp in dict_repo.keys():
    handle_packages.write(name_temp[:-10] + "\n")
handle_packages.close()

In [102]:
%%bash -s {file_packages}

wc -l $1

26 ../../conda_r_skeleton_helper/packages.txt


In [81]:
# Fork repositories
for repo in dict_repo.values():
    print(repo.full_name)
    gh_me.create_fork(repo)

conda-forge/r-ensurer-feedstock
conda-forge/r-wesanderson-feedstock
conda-forge/r-praise-feedstock
conda-forge/r-sentimentr-feedstock
conda-forge/r-infuser-feedstock
conda-forge/r-funr-feedstock
conda-forge/r-reshape-feedstock
conda-forge/r-cellranger-feedstock
conda-forge/r-alluvial-feedstock
conda-forge/r-gsalib-feedstock
conda-forge/r-etrunct-feedstock
conda-forge/r-cvxbiclustr-feedstock
conda-forge/r-labeling-feedstock
conda-forge/r-docopt-feedstock
conda-forge/r-flexdashboard-feedstock
conda-forge/r-ggedit-feedstock
conda-forge/r-rbokeh-feedstock
conda-forge/r-rlist-feedstock
conda-forge/r-flowr-feedstock
conda-forge/r-rematch-feedstock
conda-forge/r-ggparallel-feedstock
conda-forge/r-zeallot-feedstock
conda-forge/r-munsell-feedstock
conda-forge/r-waterfalls-feedstock
conda-forge/r-plotroc-feedstock
conda-forge/r-fwdselect-feedstock


In [103]:
%%bash -s {file_packages}

# Clone repositories

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  if [ ! -d "$localdir" ]
  then
    git clone --quiet git@github.com:jdblischak/${pkg}-feedstock.git "$localdir"
  fi
done

r-ensurer
r-wesanderson
r-praise
r-sentimentr
r-infuser
r-funr
r-reshape
r-cellranger
r-alluvial
r-gsalib
r-etrunct
r-cvxbiclustr
r-labeling
r-docopt
r-flexdashboard
r-ggedit
r-rbokeh
r-rlist
r-flowr
r-rematch
r-ggparallel
r-zeallot
r-munsell
r-waterfalls
r-plotroc
r-fwdselect


In [104]:
%%bash -s {dir_helper}

# Generate new recipes

# Had to manually set `--allow-archived` flag to regenerate old recipes for
# archived packages

cd "$1"
Rscript run.R

Processing r-ensurer
Adding in variants from internal_defaults
Parsing input package r-ensurer:
.. name: ensurer location: None new_location: /home/jdblischak/repos/conda_r_skeleton_helper/r-ensurer
Making/refreshing recipe for ensurer
Fetching main index from https://cran.r-project.org
Reading package metadata from /home/jdblischak/miniconda3/conda-bld/src_cache/source-ensurer_1.1_2dce76bb86.tar.gz
Writing recipe for ensurer
--dirty flag and --keep-old-work not specified. Removing build/test folder after successful build/test.

Processing r-wesanderson
Adding in variants from internal_defaults
Parsing input package r-wesanderson:
.. name: wesanderson location: None new_location: /home/jdblischak/repos/conda_r_skeleton_helper/r-wesanderson
Making/refreshing recipe for wesanderson
Fetching main index from https://cran.r-project.org
Reading package metadata from /home/jdblischak/miniconda3/conda-bld/src_cache/source-wesanderson_0.3.6_22b6ea042a.tar.gz
Writing recipe for wesanderson
--dir

INFO:conda_build.variants:Adding in variants from internal_defaults
/home/jdblischak/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cran.r-project.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/jdblischak/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cran.r-project.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
No hash (md5, sha1, sha256) provided for source-ensurer_1.1.tar.gz.  Source download forced.  Add hash to recipe to use source cache.
INFO:conda_build.config:--dirty flag and --keep-old-work not specified. Removing build/test folder after successful buil

CalledProcessError: Command 'b'\n# Generate new recipes\n\ncd "$1"\nRscript run.R\n'' returned non-zero exit status 1.

In [136]:
# Get maintainers to add back

dict_maintainers = {}

conda_forge_r = {
    "conda-forge/r",
    "johanneskoester",
    "bgruening",
    "daler",
    "jdblischak",
    "cbrueffer",
    "dbast",
    "dpryan79"
}

for name_temp in dict_repo.keys():
    file_codeowners = "/tmp/" + name_temp + "/.github/CODEOWNERS"
    handle_codeowners = open(file_codeowners, "r")
    codeowners = handle_codeowners.readlines()
    handle_codeowners.close()
    codeowners = codeowners[0][2:].replace("@", "").split(" ")
    maintainers = set(codeowners).difference(conda_forge_r)
    dict_maintainers[name_temp] = maintainers

In [181]:
%%bash -s {dir_helper}

# Copy updated recipes to cloned repositories

pkgs="$1/packages.txt"
for pkg in `cat $pkgs`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  # For some reason globbing doesn't work when run from within the notebook
  cp "$1/$pkg/meta.yaml" "$localdir/recipe/"
  cp "$1/$pkg/build.sh" "$localdir/recipe/"
  cp "$1/$pkg/bld.bat" "$localdir/recipe/"
done

r-ensurer
r-wesanderson
r-praise
r-sentimentr
r-infuser
r-funr
r-reshape
r-cellranger
r-alluvial
r-gsalib
r-etrunct
r-cvxbiclustr
r-labeling
r-docopt
r-flexdashboard
r-ggedit
r-rbokeh
r-rlist
r-flowr
r-rematch
r-ggparallel
r-zeallot
r-munsell
r-waterfalls
r-plotroc
r-fwdselect


In [182]:
# Add back maintainers

for feedstock in dict_maintainers.keys():
    maintainers = dict_maintainers[feedstock]
    if (len(maintainers)) == 0:
        continue
    print(feedstock)
    file_meta = "/tmp/" + feedstock + "/recipe/meta.yaml"
    handle_meta = open(file_meta, "r")
    meta = handle_meta.readlines()
    handle_meta.close()
    maintainers_formatted = ["    - " + id + "\n" for id in maintainers]
    lines_out = meta
    for i in range(len(meta)):
        line = meta[i]
        if line.rstrip() == "    - conda-forge/r":
            lines_out = meta[:i+1] + maintainers_formatted + meta[i+1:]
    handle_meta = open(file_meta, "w")
    handle_meta.write("".join(lines_out))
    handle_meta.close()


r-ensurer-feedstock
r-praise-feedstock
r-sentimentr-feedstock
r-infuser-feedstock
r-alluvial-feedstock
r-cvxbiclustr-feedstock
r-ggparallel-feedstock
r-zeallot-feedstock


In [183]:
%%bash -s {dir_helper}

# Change the build number to 2

pkgs="$1/packages.txt"
for pkg in `cat $pkgs`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  meta="$localdir/recipe/meta.yaml"
  sed -i s/'number: 0'/'number: 2'/ "$meta"
done

r-ensurer
r-wesanderson
r-praise
r-sentimentr
r-infuser
r-funr
r-reshape
r-cellranger
r-alluvial
r-gsalib
r-etrunct
r-cvxbiclustr
r-labeling
r-docopt
r-flexdashboard
r-ggedit
r-rbokeh
r-rlist
r-flowr
r-rematch
r-ggparallel
r-zeallot
r-munsell
r-waterfalls
r-plotroc
r-fwdselect


In [185]:
%%bash -s {file_packages}

# Commit updated recipes to branch "update-recipe"

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  git checkout -b "update-recipe"
  git add recipe/
  git commit -m "Update recipe"
done

r-ensurer
[update-recipe 82f91a0] Update recipe
 3 files changed, 30 insertions(+), 26 deletions(-)
r-wesanderson
[update-recipe c39e604] Update recipe
 3 files changed, 31 insertions(+), 25 deletions(-)
 rewrite recipe/build.sh (97%)
r-praise
[update-recipe e519306] Update recipe
 3 files changed, 25 insertions(+), 27 deletions(-)
r-sentimentr
[update-recipe 56858dc] Update recipe
 3 files changed, 36 insertions(+), 28 deletions(-)
r-infuser
[update-recipe 24aab5d] Update recipe
 3 files changed, 33 insertions(+), 28 deletions(-)
r-funr
[update-recipe 45ec4f2] Update recipe
 3 files changed, 36 insertions(+), 28 deletions(-)
r-reshape
[update-recipe 5732e39] Update recipe
 3 files changed, 28 insertions(+), 24 deletions(-)
r-cellranger
[update-recipe 17f4efc] Update recipe
 3 files changed, 31 insertions(+), 24 deletions(-)
r-alluvial
[update-recipe c3ce822] Update recipe
 3 files changed, 40 insertions(+), 29 deletions(-)
r-gsalib
[update-recipe df433ba] Update recipe
 3 files change

fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'update-recipe' already exists.
fatal: A branch named 'updat

In [186]:
%%bash -s {file_packages}

# Rerender feedstock

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  conda-smithy rerender -c auto
done

r-ensurer
Adding in variants from internal_defaults
Adding in variants from /tmp/tmpyife_qcs/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe 1062cdf] MNT: Re-rendered with conda-build 3.21.7, conda-smithy 3.16.1, and conda-forge-pinning 2021.12.30.03.08.12
 10 files changed, 34 insertions(+), 42 deletions(-)
 delete mode 100644 .ci_support/migrations/r410.yaml
r-wesanderson
Adding in variants from internal_defaults
Adding in variants from /tmp/tmp038i0a38/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe d113e3b] MNT: Re-rendered with conda-build 3.21.7, conda-smithy 3.16.1, and conda-forge-pinning 2021.12.30.03.08.12
 10 files changed, 34 insertions(+), 43 deletions(-)
 delete mode 100644 .ci_support/migrations/r410.yaml
r-praise
Adding in variants from internal_defaults
Adding in variants from /tmp/tmpo_t_cuxi/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe b639f92] MNT: Re-rendered with 

INFO:conda_smithy.configure_feedstock:Downloading conda-forge-pinning-2021.12.30.03.08.12
INFO:conda_smithy.configure_feedstock:Extracting conda-forge-pinning to /tmp/tmpyife_qcs
INFO:conda_smithy.configure_feedstock:r410.yaml is closed now. Removing
Setting build platform. This is only useful when pretending to be on another platform, such as for rendering necessary dependencies on a non-native platform. I trust that you know what you're doing.
Setting build arch. This is only useful when pretending to be on another arch, such as for rendering necessary dependencies on a non-native arch. I trust that you know what you're doing.
No numpy version specified in conda_build_config.yaml.  Falling back to default numpy value of 1.16
INFO:conda_build.variants:Adding in variants from internal_defaults
INFO:conda_build.variants:Adding in variants from /tmp/tmpyife_qcs/conda_build_config.yaml
INFO:conda_build.variants:Adding in variants from argument_variants
/home/jdblischak/miniconda3/lib/pyth

In [187]:
%%bash -s {file_packages}

# Push to GitHub

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  git push origin update-recipe
done

r-ensurer
r-wesanderson
r-praise
r-sentimentr
r-infuser
r-funr
r-reshape
r-cellranger
r-alluvial
r-gsalib
r-etrunct
r-cvxbiclustr
r-labeling
r-docopt
r-flexdashboard
r-ggedit
r-rbokeh
r-rlist
r-flowr
r-rematch
r-ggparallel
r-zeallot
r-munsell
r-waterfalls
r-plotroc
r-fwdselect


remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-ensurer-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-ensurer-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-wesanderson-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-wesanderson-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-praise-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-praise-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://g

In [192]:
# Open pull requests
# https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html?highlight=create_pull#github.Repository.Repository.create_pull
# https://docs.github.com/en/rest/reference/pulls#create-a-pull-request
# https://pygithub.readthedocs.io/en/latest/examples/PullRequest.html?highlight=create%20a%20pull%20request#create-a-new-pull-request
for repo in dict_repo.values():
    print(repo.full_name)
    if repo.full_name == "conda-forge/r-ensurer-feedstock":
        continue
    body = """Checklist
* [x] Used a [personal fork of the feedstock to propose changes](https://conda-forge.org/docs/maintainer/updating_pkgs.html#forking-and-pull-requests)
* [x] Bumped the build number (if the version is unchanged)
* [ ] Reset the build number to `0` (if the version changed)
* [x] [Re-rendered]( https://conda-forge.org/docs/maintainer/updating_pkgs.html#rerendering-feedstocks ) with the latest `conda-smithy` (Use the phrase <code>@<space/>conda-forge-admin, please rerender</code> in a comment in this PR for automated rerendering)
* [x] Ensured the license file is being packaged.

* I already rerendered locally
* I bumped the build number to 2 as an intial guess. I'll fix it if necessary
"""
    repo.create_pull(title="Update recipe", body=body, base="master", head="jdblischak:update-recipe")

conda-forge/r-ensurer-feedstock


GithubException: 422 {"message": "Validation Failed", "errors": [{"resource": "PullRequest", "code": "custom", "message": "A pull request already exists for jdblischak:update-recipe."}], "documentation_url": "https://docs.github.com/rest/reference/pulls#create-a-pull-request"}